##ENTORNO

In [1]:
#Instalar dependencias para instalar Google-Chrome
!sudo apt-get update
!sudo apt-get install -y libu2f-udev
!pip install selenium

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:11 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,252 kB]
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,351 kB]
Get:14 http://secu

In [2]:
#Instalar Google Chrome
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get install -f
!rm -rf google-chrome-stable_current_amd64.deb

--2023-06-01 18:33:20--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 108.177.119.136, 108.177.119.190, 108.177.119.91, ...
Connecting to dl.google.com (dl.google.com)|108.177.119.136|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93841272 (89M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb’

google-chrome-stabl 100%[===================>]  89.49M   237MB/s    in 0.4s    

2023-06-01 18:33:20 (237 MB/s) - ‘google-chrome-stable_current_amd64.deb’ saved [93841272/93841272]

Selecting previously unselected package google-chrome-stable.
(Reading database ... 122659 files and directories currently installed.)
Preparing to unpack google-chrome-stable_current_amd64.deb ...
Unpacking google-chrome-stable (114.0.5735.90-1) ...
Setting up google-chrome-stable (114.0.5735.90-1) ...
update-alternatives: using /usr/bin/google-chrome-stable to provide /usr/bin/x-www-b

In [3]:
# Descargar WebDriver
!wget https://chromedriver.storage.googleapis.com/113.0.5672.24/chromedriver_linux64.zip
!unzip chromedriver_linux64
!rm -rf chromedriver_linux64.zip
!rm -rf LICENSE.chromedriver

--2023-06-01 18:33:38--  https://chromedriver.storage.googleapis.com/113.0.5672.24/chromedriver_linux64.zip
Resolving chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)... 108.177.126.128, 2a00:1450:4013:c01::80
Connecting to chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)|108.177.126.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7172785 (6.8M) [application/zip]
Saving to: ‘chromedriver_linux64.zip’

chromedriver_linux6 100%[===================>]   6.84M  9.05MB/s    in 0.8s    

2023-06-01 18:33:39 (9.05 MB/s) - ‘chromedriver_linux64.zip’ saved [7172785/7172785]

Archive:  chromedriver_linux64.zip
  inflating: chromedriver            
  inflating: LICENSE.chromedriver    


##PAGINA A INSPECCIONAR

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Opciones de Chrome
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')

# Ruta del controlador de Chrome
chrome_driver_path = '/content/chromedriver'

# Crear un objeto Service para especificar la ruta del controlador
service = Service(chrome_driver_path)

# Crear una instancia del navegador Chrome
driver = webdriver.Chrome(service=service, options=chrome_options)

# Navegar a una página web
driver.get('https://tablademareas.com/cl/los-lagos/puerto-montt')
#driver.get('https://tablademareas.com/cl/magallanes/punta-arenas')

# Obtener el HTML de la página después de activar el evento onclick
html = driver.page_source

# Crear un objeto BeautifulSoup para analizar el HTML
soup = BeautifulSoup(html, 'html.parser')

# Navegar a una página web
driver.get('https://tablademareas.com/cl/los-lagos/puerto-montt')
#driver.get('https://tablademareas.com/cl/magallanes/punta-arenas')

wait = WebDriverWait(driver, 10)
element = wait.until(EC.presence_of_element_located((By.ID, 'selector_api_marine')))

# Activar el evento onclick del elemento
driver.execute_script("arguments[0].click();", element)

# Obtener el HTML de la página después de activar el evento onclick
html1 = driver.page_source

# Crear un objeto BeautifulSoup para analizar el HTML
soup1 = BeautifulSoup(html1, 'html.parser')

##DATOS CLIMATOLOGICOS

In [5]:
def buscar_elemento(soup, tag, identificador):
    elemento = soup.find(tag, id=identificador) or soup.find(tag, class_=identificador)
    if elemento:
        return elemento.text.strip()
    else:
        return f'{tag.capitalize()} no encontrado'

In [6]:
## Función para buscar color en Descenso /Estable/ En Ascenso
def buscar_elemento_v(soup, tag, identificador):
    elemento = soup.find(tag, id=identificador) or soup.find(tag, class_=identificador)
    if elemento:
        return elemento
    else:
        return None

In [7]:
## Retorno de estado_color hacia estado
def obtener_estado(Presion_Actividad_Pesca, ascenso_color, estable_color, descenso_color):
    if Presion_Actividad_Pesca == 'ascenso_color':
        resultado = 'en ascenso'
    elif Presion_Actividad_Pesca == 'estable_color':
        resultado = 'estable'
    elif Presion_Actividad_Pesca == 'descenso_color':
        resultado = 'en descenso'
    else:
        resultado = "Presión de actividad de pesca desconocida"

    return resultado

In [8]:
## Retorno mensaje de pesca
def obtener_mensaje(Presion_Actividad_Pesca, ascenso_color, estable_color, descenso_color):
    Mensaje = [
        'Muy buena señal. Actividad alta prevista',
        'Actividad normal de los peces',
        'Buena pesca al principio, pero pronto dejarán de alimentarse'
    ]

    if Presion_Actividad_Pesca == 'ascenso_color':
        resultado = Mensaje[0]
    elif Presion_Actividad_Pesca == 'estable_color':
        resultado = Mensaje[1]
    elif Presion_Actividad_Pesca == 'descenso_color':
        resultado = Mensaje[2]
    else:
        resultado = "Presión de actividad de pesca desconocida"

    return resultado

In [9]:
## Retorno del valor activo Descenso /Estable/ En Ascenso
def encontrar_activa_v2(x, y, z):
    variables = {'ascenso_color': ascenso_color, 'estable_color': estable_color, 'descenso_color': descenso_color}
    activas = [variable for variable, valor in variables.items() if valor != '#ffffff']
    if len(activas) == 1:
        return activas[0]
    else:
        return "Ninguna o multiples variables activas"

In [10]:
## Estado para la pesca
def obtener_estado_activo_pesca(soup):
    elementos = soup.find_all('div', class_='circulo_estado_grafico_barometro1')
    
    for elemento in elementos:
        if 'circulo_estado_grafico_barometro1_activo' in elemento['class']:
            return elemento.next_sibling.strip()
    
    return "Estado pesca actual no encontrado"

In [11]:
#Viento
Viento = buscar_elemento(soup, 'span', 'velocidad_grafico_viento')
Viento_unidades = buscar_elemento(soup, 'span', 'unidades_grafico_viento')
Rachas = buscar_elemento(soup, 'span', 'rafagas_datos_grafico_tiempo_valor')
print('Viento:',Viento)
print('Unidades de viento:',Viento_unidades,)
print('Procedencia_Viento:',Rachas)
print('\n')

#Section Temperatura
Temperatura = buscar_elemento(soup, 'span', 'temperatura_grafico_termometro')
Temperatura_Unidad = '°C'
Temperatura_Max = buscar_elemento(soup, 'span', 'temperatura_grafico_termometro_max')
Temperatura_Unidad_Max = '°C'
Temperatura_Min = buscar_elemento(soup, 'span', 'temperatura_grafico_termometro_min')
Temperatura_Unidad_Min = '°C'
Temperatura_Sensacion = buscar_elemento(soup, 'span', 'temperatura_grafico_termometro_sensacion')
Temperatura_Unidad_Sensacion = '°C'

print('Temperatura:',Temperatura)
print('Unidad de Temperatura:',Temperatura_Unidad)
print('Temperatura Max:',Temperatura_Max)
print('Unidad de Temperatura Max:',Temperatura_Unidad_Max)
print('Temperatura Min:',Temperatura_Min)
print('Unidad de Temperatura Min:',Temperatura_Unidad_Min)
print('Temperatura Sensacion:',Temperatura_Sensacion)
print('Unidad de Temperatura Sensacion:',Temperatura_Unidad_Sensacion)


print('\n')

# Humedad
Humedad = buscar_elemento(soup, 'span', 'humedad_grafico_humedad')
Unidad_humedad = '%'
Pto_Rocio = buscar_elemento(soup, 'span', 'rocio_datos_grafico_tiempo_valor')
Unidad_Pto_Rocio = '°C'

print('Humedad:',Humedad)
print('Unidad Humedad:',Unidad_humedad)
print('Punto de Rocio:',Pto_Rocio)
print('Unidad Punto de Rocio:',Unidad_Pto_Rocio)
print('\n')

#Tiempo
Visibilidad = buscar_elemento(soup, 'span', 'numero_visibilidad_grafico_visibilidad')
Visibilidad_Unidad = buscar_elemento(soup, 'span', 'unidades_grafico_viento')

print('Visibilidad:',Visibilidad)
print('Visibilidad Unidad:',Visibilidad_Unidad)
print('\n')

#Presion
Presion = buscar_elemento(soup, 'span', 'numeros_presion_grafico_barometro')
Unidad_Presion = buscar_elemento(soup, 'span', 'unidades_presion_grafico_barometro')

print('Presion:',Presion)
print('Unidad Presion:',Unidad_Presion)
print('\n')

## Presión en la actividad pesquera

# Obtener el color de fondo
ascenso_elemento = buscar_elemento_v(soup, 'div', 'leyenda_influencia_grafico_barometro_ascenso')
estable_elemento = buscar_elemento_v(soup, 'div', 'leyenda_influencia_grafico_barometro_estable')
descenso_elemento = buscar_elemento_v(soup, 'div', 'leyenda_influencia_grafico_barometro_descenso')

# Obtener el color de fondo
ascenso_color = ascenso_elemento['style'].split(':')[1].strip()
estable_color = estable_elemento['style'].split(':')[1].strip()
descenso_color = descenso_elemento['style'].split(':')[1].strip()

Color_Presion_Actividad_Pesca=encontrar_activa_v2(ascenso_color,estable_color,descenso_color)     
Presion_Actividad_Pesca=obtener_estado(Color_Presion_Actividad_Pesca, ascenso_color, estable_color, descenso_color)
Mensaje_Presion_Actividad_Pesca = obtener_mensaje(Color_Presion_Actividad_Pesca, ascenso_color, estable_color, descenso_color)
Estado_activo_pesca=obtener_estado_activo_pesca(soup)

print('Color Actividad Pesca:',Color_Presion_Actividad_Pesca)
print('Presion Actividad Pesca:',Presion_Actividad_Pesca)
print('Mensaje Presion Actividad Pesca:',Mensaje_Presion_Actividad_Pesca)
print('Estado activo pesca:',Estado_activo_pesca)

print('\n')


Viento: 24
Unidades de viento: km/h
Procedencia_Viento: 32 km/h


Temperatura: 13
Unidad de Temperatura: °C
Temperatura Max: 14
Unidad de Temperatura Max: °C
Temperatura Min: 7
Unidad de Temperatura Min: °C
Temperatura Sensacion: 12
Unidad de Temperatura Sensacion: °C


Humedad: 70
Unidad Humedad: %
Punto de Rocio: 8
Unidad Punto de Rocio: °C


Visibilidad: 15
Visibilidad Unidad: km/h


Presion: 1003
Unidad Presion: hPa


Color Actividad Pesca: descenso_color
Presion Actividad Pesca: en descenso
Mensaje Presion Actividad Pesca: Buena pesca al principio, pero pronto dejarán de alimentarse
Estado activo pesca: Estado pesca actual no encontrado




## TABLAS MAREAS

In [12]:
def tabla_zona_costa_captura_datos(soup, id_fila):
    tabla = soup.find('div', id=f'tabla_datos_tiempo_datos_datos_cuadro_pie_{id_fila}')
    numero_lluvia = None
    cantidad_lluvia = None
    
    if tabla is not None:
        numero_lluvia_element = tabla.find('span', class_='tabla_datos_tiempo_datos_datos_cuadro_pie_color_lila')
        cantidad_lluvia_element = tabla.find('span', class_='tabla_datos_tiempo_datos_datos_cuadro_pie_color_celeste')
        
        if numero_lluvia_element is not None:
            numero_lluvia = numero_lluvia_element.text
        
        if cantidad_lluvia_element is not None:
            cantidad_lluvia = cantidad_lluvia_element.text

    return numero_lluvia, cantidad_lluvia

In [13]:
def tabla_zona_api_marine(soup1, id_fila):
  # Buscar el elemento padre por su ID
  etiqueta_padre_marine = soup.find('div', id='selector_api')

  # Activar estado de zona de costa
  etiqueta_zona_costa_marine = etiqueta_padre_marine.find('div', id='selector_api_marine')

  # Asignar el nuevo valor al atributo onclick
  etiqueta_zona_costa_marine['onclick'] = "CambiarAPI('marine', 1);"

  # Obtener el contenido principal de zona costera
  contenido_zona_costa_marine = etiqueta_zona_costa_marine.find('div', class_='selector_api_texto').text

  # Imprimir el resultado
  print(contenido_zona_costa_marine)

  contenedor_principal = soup1.find('div', id='tabla_datos_tiempo_datos')
  fila_datos = contenedor_principal.find('div', id=f'tabla_datos_tiempo_datos_{id_fila}')
  tiempo = fila_datos.find('div', class_='tabla_datos_tiempo_datos_tiempo').text
  datos_tiempo = fila_datos.find('div', class_='tabla_datos_tiempo_datos_datos')
  cuadros = datos_tiempo.find_all('div', class_='tabla_datos_tiempo_datos_datos_cuadro')
  resultados = []

  humedad_encontrada = False  # Variable para verificar si se ha encontrado una entrada de humedad válida

  for cuadro in cuadros:
      titulo_element = cuadro.find('div', class_='tabla_datos_tiempo_datos_datos_cuadro_titulo')
      numero_element = cuadro.find('div', class_='tabla_datos_tiempo_datos_datos_cuadro_num')
      unidades_element = cuadro.find('span', class_='tabla_datos_tiempo_datos_datos_cuadro_num_unidades')

      titulo = titulo_element.text if titulo_element else ''
      numero = numero_element.text if numero_element else ''
      unidades = unidades_element.text if unidades_element else ''

      # Omitir los resultados con título vacío
      if not titulo:
          continue

      # Solución para capturar el valor de temperatura correctamente
      if titulo == 'Temp.':
          numero_element = cuadro.find('span', class_='tabla_datos_tiempo_datos_datos_cuadro_num_grande_color')
          numero = numero_element.text if numero_element else ''

      # Solución para capturar el valor de viento correctamente
      if titulo == 'Viento':
          viento_valor_element = cuadro.find('div', class_='tabla_datos_tiempo_datos_datos_cuadro_flecha_numero')
          viento_valor = viento_valor_element.text if viento_valor_element else ''
          resultado = {
              'titulo': titulo,
              'numero': viento_valor,
              'unidades': ''
          }
          resultados.append(resultado)
          continue

      resultado = {
          'titulo': titulo,
          'numero': numero,
          'unidades': unidades
      }
      
      if titulo == 'Humedad' and not humedad_encontrada:
          humedad_numero_element = cuadro.find('div', class_='tabla_datos_tiempo_datos_datos_cuadro_num_grande')
          humedad_numero = humedad_numero_element.find('span', class_='tabla_datos_tiempo_datos_datos_cuadro_num_numero').text
          humedad_unidades_element = cuadro.find('span', class_='tabla_datos_tiempo_datos_datos_cuadro_num_unidades')
          humedad_unidades = humedad_unidades_element.text if humedad_unidades_element else ''
          if humedad_numero and humedad_unidades:
              resultado_humedad = {
                  'titulo': titulo,
                  'numero': humedad_numero,
                  'unidades': humedad_unidades
              }
              resultados.append(resultado_humedad)
              humedad_encontrada = True  # Marcar la humedad como encontrada
      else:
          resultados.append(resultado)

  presion_influencia_element = contenedor_principal.find('div', class_='tabla_datos_tiempo_datos_datos_presion_influencia')
  if presion_influencia_element is not None:
      presion_influencia = presion_influencia_element.text.strip()
      resultados.append({'Presión Influencia': presion_influencia})

  presion_estado_element = contenedor_principal.find('div', class_='tabla_datos_tiempo_datos_datos_presion_estado')
  if presion_estado_element is not None:
      presion_estado = presion_estado_element.text.strip()
      resultados.append({'Presión Estado': presion_estado})

  sensacion_element = contenedor_principal.find('div', class_='tabla_datos_tiempo_datos_datos_cuadro_subtitulo')
  if sensacion_element is not None:
      sensacion_valor_element = sensacion_element.find_next_sibling('div', class_='tabla_datos_tiempo_datos_datos_cuadro_num_chico')
      if sensacion_valor_element is not None:
          sensacion_valor = sensacion_valor_element.text.strip()
          resultados.append({'Sensación': sensacion_valor})

  punto_rocio_element = contenedor_principal.find('div', class_='tabla_datos_tiempo_datos_datos_cuadro_subtitulo', string='punto rocío')
  if punto_rocio_element is not None:
      punto_rocio_valor_element = punto_rocio_element.find_next_sibling('div', class_='tabla_datos_tiempo_datos_datos_cuadro_num_chico')
      if punto_rocio_valor_element is not None:
          punto_rocio_valor = punto_rocio_valor_element.text.strip()
          resultados.append({'Punto Rocío': punto_rocio_valor})

  viento_element = contenedor_principal.find('div', class_='tabla_datos_tiempo_datos_datos_cuadro_num_grande', title='Velocidad del viento (km/h)\ny procedencia del viento')
  if viento_element is not None:
      viento_valor_element = viento_element.find_next_sibling('div', class_='tabla_datos_tiempo_datos_datos_cuadro_flecha_numero')
      if viento_valor_element is not None:
          viento_valor = viento_valor_element.text.strip()
          resultados.append({'Viento': viento_valor})

  direccion_element = contenedor_principal.find('div', class_='tabla_datos_tiempo_datos_datos_cuadro_num_direccion')
  if direccion_element is not None:
      direccion_texto = direccion_element.text.strip()
      viento_resultado = next((r for r in resultados if r['titulo'] == 'Viento'), None)
      if viento_resultado:
          viento_resultado['unidades'] = direccion_texto

  resultados = [r for r in resultados if r.get('titulo') != 'Humidity']
  
  hora_element = fila_datos.find('div', class_='tabla_datos_tiempo_datos_tiempo_hora')
  numero_hora = hora_element.find('span', class_='tabla_datos_tiempo_datos_tiempo_hora_num').text
  letra_hora = hora_element.text.strip()[-1]
  
  numero_lluvia, cantidad_lluvia = tabla_zona_costa_captura_datos(soup, id_fila)
  
  resultados.append({'Hora': f'{numero_hora} {letra_hora}'})
  resultados.append({'Probabilidad de lluvia': numero_lluvia})
  resultados.append({'Cantidad de agua': cantidad_lluvia})
  
  return resultados

In [14]:
def tabla_zona_api_weather(soup, id_fila):
# Buscar el elemento padre por su ID
  etiqueta_padre_marine = soup.find('div', id='selector_api')

  # Activar estado de zona de costa
  etiqueta_zona_costa_marine = etiqueta_padre_marine.find('div', id='selector_api_weather')

  # Asignar el nuevo valor al atributo onclick
  etiqueta_zona_costa_marine['onclick'] = "CambiarAPI('weather', 1);"

  # Obtener el contenido principal de zona costera
  contenido_zona_costa_marine = etiqueta_zona_costa_marine.find('div', class_='selector_api_texto').text

  # Imprimir el resultado
  print(contenido_zona_costa_marine)

  contenedor_principal = soup.find('div', id='tabla_datos_tiempo_datos')
  fila_datos = contenedor_principal.find('div', id=f'tabla_datos_tiempo_datos_{id_fila}')
  tiempo = fila_datos.find('div', class_='tabla_datos_tiempo_datos_tiempo').text
  datos_tiempo = fila_datos.find('div', class_='tabla_datos_tiempo_datos_datos')
  cuadros = datos_tiempo.find_all('div', class_='tabla_datos_tiempo_datos_datos_cuadro')
  resultados = []

  humedad_encontrada = False  # Variable para verificar si se ha encontrado una entrada de humedad válida

  for cuadro in cuadros:
      titulo_element = cuadro.find('div', class_='tabla_datos_tiempo_datos_datos_cuadro_titulo')
      numero_element = cuadro.find('div', class_='tabla_datos_tiempo_datos_datos_cuadro_num')
      unidades_element = cuadro.find('span', class_='tabla_datos_tiempo_datos_datos_cuadro_num_unidades')

      titulo = titulo_element.text if titulo_element else ''
      numero = numero_element.text if numero_element else ''
      unidades = unidades_element.text if unidades_element else ''

      # Omitir los resultados con título vacío
      if not titulo:
          continue

      # Solución para capturar el valor de temperatura correctamente
      if titulo == 'Temp.':
          numero_element = cuadro.find('span', class_='tabla_datos_tiempo_datos_datos_cuadro_num_grande_color')
          numero = numero_element.text if numero_element else ''

      # Solución para capturar el valor de viento correctamente
      if titulo == 'Viento':
          viento_valor_element = cuadro.find('div', class_='tabla_datos_tiempo_datos_datos_cuadro_flecha_numero')
          viento_valor = viento_valor_element.text if viento_valor_element else ''
          resultado = {
              'titulo': titulo,
              'numero': viento_valor,
              'unidades': ''
          }
          resultados.append(resultado)
          continue

      resultado = {
          'titulo': titulo,
          'numero': numero,
          'unidades': unidades
      }
      
      if titulo == 'Humedad' and not humedad_encontrada:
          humedad_numero_element = cuadro.find('div', class_='tabla_datos_tiempo_datos_datos_cuadro_num_grande')
          humedad_numero = humedad_numero_element.find('span', class_='tabla_datos_tiempo_datos_datos_cuadro_num_numero').text
          humedad_unidades_element = cuadro.find('span', class_='tabla_datos_tiempo_datos_datos_cuadro_num_unidades')
          humedad_unidades = humedad_unidades_element.text if humedad_unidades_element else ''
          if humedad_numero and humedad_unidades:
              resultado_humedad = {
                  'titulo': titulo,
                  'numero': humedad_numero,
                  'unidades': humedad_unidades
              }
              resultados.append(resultado_humedad)
              humedad_encontrada = True  # Marcar la humedad como encontrada
      else:
          resultados.append(resultado)

  presion_influencia_element = contenedor_principal.find('div', class_='tabla_datos_tiempo_datos_datos_presion_influencia')
  if presion_influencia_element is not None:
      presion_influencia = presion_influencia_element.text.strip()
      resultados.append({'Presión Influencia': presion_influencia})

  presion_estado_element = contenedor_principal.find('div', class_='tabla_datos_tiempo_datos_datos_presion_estado')
  if presion_estado_element is not None:
      presion_estado = presion_estado_element.text.strip()
      resultados.append({'Presión Estado': presion_estado})

  sensacion_element = contenedor_principal.find('div', class_='tabla_datos_tiempo_datos_datos_cuadro_subtitulo')
  if sensacion_element is not None:
      sensacion_valor_element = sensacion_element.find_next_sibling('div', class_='tabla_datos_tiempo_datos_datos_cuadro_num_chico')
      if sensacion_valor_element is not None:
          sensacion_valor = sensacion_valor_element.text.strip()
          resultados.append({'Sensación': sensacion_valor})

  punto_rocio_element = contenedor_principal.find('div', class_='tabla_datos_tiempo_datos_datos_cuadro_subtitulo', string='punto rocío')
  if punto_rocio_element is not None:
      punto_rocio_valor_element = punto_rocio_element.find_next_sibling('div', class_='tabla_datos_tiempo_datos_datos_cuadro_num_chico')
      if punto_rocio_valor_element is not None:
          punto_rocio_valor = punto_rocio_valor_element.text.strip()
          resultados.append({'Punto Rocío': punto_rocio_valor})

  viento_element = contenedor_principal.find('div', class_='tabla_datos_tiempo_datos_datos_cuadro_num_grande', title='Velocidad del viento (km/h)\ny procedencia del viento')
  if viento_element is not None:
      viento_valor_element = viento_element.find_next_sibling('div', class_='tabla_datos_tiempo_datos_datos_cuadro_flecha_numero')
      if viento_valor_element is not None:
          viento_valor = viento_valor_element.text.strip()
          resultados.append({'Viento': viento_valor})

  direccion_element = contenedor_principal.find('div', class_='tabla_datos_tiempo_datos_datos_cuadro_num_direccion')
  if direccion_element is not None:
      direccion_texto = direccion_element.text.strip()
      viento_resultado = next((r for r in resultados if r['titulo'] == 'Viento'), None)
      if viento_resultado:
          viento_resultado['unidades'] = direccion_texto

  resultados = [r for r in resultados if r.get('titulo') != 'Humidity']
  
  hora_element = fila_datos.find('div', class_='tabla_datos_tiempo_datos_tiempo_hora')
  numero_hora = hora_element.find('span', class_='tabla_datos_tiempo_datos_tiempo_hora_num').text
  letra_hora = hora_element.text.strip()[-1]
  
  numero_lluvia, cantidad_lluvia = tabla_zona_costa_captura_datos(soup, id_fila)
  
  resultados.append({'Hora': f'{numero_hora} {letra_hora}'})
  resultados.append({'Probabilidad de lluvia': numero_lluvia})
  resultados.append({'Cantidad de agua': cantidad_lluvia})
  
  return resultados

In [15]:
def imprimir_tablas(soup):
  for id_fila in range(24):
    if (soup is globals().get('soup')):
      resultados = tabla_zona_api_weather(soup, id_fila)
      print(resultados)
    elif (soup is globals().get('soup1')):
      resultados = tabla_zona_api_marine(soup, id_fila)
      print(resultados)

In [16]:
imprimir_tablas(soup)


 Previsión en zona costera 
 COSTA DE PUERTO MONTT 

[{'titulo': 'Presión', 'numero': '1017', 'unidades': ''}, {'titulo': 'Temp.', 'numero': '8°', 'unidades': ' °C'}, {'titulo': 'Humedad', 'numero': '93', 'unidades': ' %'}, {'titulo': 'Viento', 'numero': '8', 'unidades': 'NNE'}, {'Presión Influencia': 'en descenso'}, {'Presión Estado': 'Muy bueno'}, {'Sensación': '6 °C'}, {'Punto Rocío': '7 °C'}, {'Hora': '0 h'}, {'Probabilidad de lluvia': None}, {'Cantidad de agua': None}]

 Previsión en zona costera 
 COSTA DE PUERTO MONTT 

[{'titulo': 'Presión', 'numero': '1016', 'unidades': ''}, {'titulo': 'Temp.', 'numero': '8°', 'unidades': ' °C'}, {'titulo': 'Humedad', 'numero': '93', 'unidades': ' %'}, {'titulo': 'Viento', 'numero': '8', 'unidades': 'NNE'}, {'Presión Influencia': 'en descenso'}, {'Presión Estado': 'Muy bueno'}, {'Sensación': '6 °C'}, {'Punto Rocío': '7 °C'}, {'Hora': '1 h'}, {'Probabilidad de lluvia': None}, {'Cantidad de agua': None}]

 Previsión en zona costera 
 COSTA DE P

In [17]:
imprimir_tablas(soup1)


 Previsión en aguas abiertas 
 MAR EN PUERTO MONTT 

[{'titulo': 'Presión', 'numero': '1015', 'unidades': ''}, {'titulo': 'Temp.', 'numero': '10°', 'unidades': ' °C'}, {'titulo': 'Humedad', 'numero': '84', 'unidades': ' %'}, {'titulo': 'Viento', 'numero': '13', 'unidades': 'NNE'}, {'Presión Influencia': 'en descenso'}, {'Presión Estado': 'Muy bueno'}, {'Sensación': '8 °C'}, {'Punto Rocío': '8 °C'}, {'Hora': '0 h'}, {'Probabilidad de lluvia': None}, {'Cantidad de agua': None}]

 Previsión en aguas abiertas 
 MAR EN PUERTO MONTT 

[{'titulo': 'Presión', 'numero': '1014', 'unidades': ''}, {'titulo': 'Temp.', 'numero': '10°', 'unidades': ' °C'}, {'titulo': 'Humedad', 'numero': '83', 'unidades': ' %'}, {'titulo': 'Viento', 'numero': '14', 'unidades': 'NNE'}, {'Presión Influencia': 'en descenso'}, {'Presión Estado': 'Muy bueno'}, {'Sensación': '8 °C'}, {'Punto Rocío': '8 °C'}, {'Hora': '1 h'}, {'Probabilidad de lluvia': None}, {'Cantidad de agua': None}]

 Previsión en aguas abiertas 
 MAR 

## INDICE ULTRAVIOLETA

In [18]:
def mensaje_de_seguridad(uv_maximo_img_num):
   Mensaje = ['Puedes salir sin necesidad de tomar ninguna medida de protección solar.', 'Viste camiseta, gorra y gafas. \n Usa protección solar 30+. \n Mantente a la sombra en las horas centrales del día.', 'Viste camiseta, gorra y gafas. \n Usa protección solar 50+. \n Mantente a la sombra todo el tiempo posible y evita salir en las horas centrales del día.']
   if uv_maximo_img_num <3:
    resultado = Mensaje[0]
   elif (uv_maximo_img_num >3) & (uv_maximo_img_num <8) :
    resultado = Mensaje[1]
   elif uv_maximo_img_num >=8:
    resultado = Mensaje[2]
   else:
    resultado = "Recomendacion para indice ultravioleta no encontrado"

   return resultado


In [19]:
uv_maximo_texto = soup.find('div', {'id': 'uv_maximo_texto'}).text

# Obtener el número 2 del div con id="uv_maximo_img_num"
uv_maximo_img_num = soup.find('div', {'id': 'uv_maximo_img_num'}).text

# Imprimir los resultados
print('Indice ultravioleta:', uv_maximo_texto)
print('Valor de indice ultravioleta:', uv_maximo_img_num)
print('Mensaje respecto al indice ultravioleta:',mensaje_de_seguridad(int(uv_maximo_img_num)))


Indice ultravioleta: bajo
Valor de indice ultravioleta: 2
Mensaje respecto al indice ultravioleta: Puedes salir sin necesidad de tomar ninguna medida de protección solar.


## TEMPERATURA DEL AGUA

In [20]:
actual_element = soup.find('span', {'id': 'c_grafico_temp_agua_txt_grados_agua_actual'})

# Extraer el valor del elemento actual
temp_actual_agua = actual_element.text.strip()

# Encontrar el elemento con id "c_grafico_temp_agua_txt_grados_agua_media"
media_element = soup.find('span', {'id': 'c_grafico_temp_agua_txt_grados_agua_media'})

# Extraer el valor del elemento de la media
temp_media_agua = media_element.text.strip()

# Imprimir los valores
print("Temperatura actual del agua:", temp_actual_agua)
print("Temperatura media del agua:", temp_media_agua)

Temperatura actual del agua: 11°C
Temperatura media del agua: 10°C


In [21]:
rafificacion_promedio=0
valores_horarios_valor = []
valores_horarios = []
horas = soup.find_all('div', {'class': 'evolucion_temp_agua_hora'})
for i, hora in zip(range(14),horas):
  id_valor_relleno = f"evolucion_temp_agua_relleno_cuadro_{i}"
  valor_incremental = soup.find('div', {'class': 'evolucion_temp_agua_relleno_cuadro', 'id': id_valor_relleno}).text.strip()
  valor_horario = hora.text.strip()
  valores_horarios.append(valor_horario)
  valores_horarios_valor.append(valor_incremental)
for valor, valor1 in zip(valores_horarios_valor,valores_horarios):
  print(valor1, valor)
  numero = ''.join(filter(str.isdigit, valor))
  rafificacion_promedio=rafificacion_promedio+int(numero)
print(rafificacion_promedio/13)

0 h 8°
1 h 11°
2 h 11°
3 h 8°
4 h 11°
5 h 11°
6 h 8°
7 h 11°
8 h 11°
9 h 8°
10 h 11°
11 h 11°
12 h 8°
13 h 11°
10.692307692307692


## OLEAJE

In [22]:
valor_fecha = soup.find('div', class_='texto_grafico_estado_actual_fecha').text.strip()
print("Valor de la fecha:", valor_fecha)

direccion_de_oleaje = soup.find('span', id='swellDir16Point').text
print("Direccion de oleaje:", direccion_de_oleaje)

valor_de_oleaje = soup.find('span', id='swellDir').text
print("Valor de oleaje:", valor_de_oleaje)

altura_significativa = soup.find('span', id='sigHeight_m').text
print("Altura significativa de las olas:", altura_significativa)

periodo_olas = soup.find('span', id='swellPeriod_secs').text
print("Periodo de las olas:", periodo_olas)

valor_costa = soup.find('span', class_='grafico_viento_oleaje_actual_titulo_txt_costa').text.strip()
valor_direccion = soup.find('span', id='oleaje_viento_direccion_txt').text.strip()
valor_grados = soup.find('span', id='oleaje_viento_direccion_grados').text.strip()
valor_velocidad = soup.find('span', id='oleaje_viento_velocidad').text.strip()

print("Valor de costa:", valor_costa)
print("Valor de dirección:", valor_direccion)
print("Valor de grados:", valor_grados)
print("Valor de velocidad:", valor_velocidad)

valor_aguas_abiertas = soup.find('span', class_='grafico_viento_oleaje_actual_titulo_txt_mar').text.strip()
valor_direccion = soup.find('span', id='oleaje_viento_marine_direccion_txt').text.strip()
valor_grados = soup.find('span', id='oleaje_viento_marine_direccion_grados').text.strip()
valor_velocidad = soup.find('span', id='oleaje_viento_marine_velocidad').text.strip()

print("Valor de aguas abiertas:", valor_aguas_abiertas)
print("Valor de dirección:", valor_direccion)
print("Valor de grados:", valor_grados)
print("Valor de velocidad:", valor_velocidad)


alturas_oleaje_actual = soup.find_all('div', class_='alturas_oleaje_actual')
for elemento in alturas_oleaje_actual:
    titulo = elemento.find('div', class_='alturas_oleaje_actual_titulo').text.strip()
    texto = elemento.find('div', class_='alturas_oleaje_actual_texto').text.strip()
    valor = elemento.find('div', class_='circulo_alturas_oleaje').text.strip()
    print("Título:", titulo)
    print("Texto:", texto)
    print("Valor:", valor)


Valor de la fecha: 1 DE JUNIO DE 2023, 14:33 h
Direccion de oleaje: NE
Valor de oleaje: 50
Altura significativa de las olas: 0.3 m
Periodo de las olas: 0.1 s
Valor de costa: costa
Valor de dirección: NNE
Valor de grados: 15
Valor de velocidad: 24 km/h
Valor de aguas abiertas: aguas abiertas
Valor de dirección: NNE
Valor de grados: 25
Valor de velocidad: 21 km/h
Título: Altura más frecuente
Texto: La mayoría de las olas tendrán la mitad de la altura significativa.
Valor: 0.1 m
Título: Altura significativa
Texto: Aprox. el 14% de las olas serán más grandes que la altura significativa (aprox. 1 de cada 7 olas).
Valor: 0.3 m
Título: Altura máxima
Texto: Lo normal es esperar una ola del doble de la altura significativa 3 veces cada 24 horas.
Valor: 0.6 m


In [23]:
horas = [elemento.text for elemento in soup.find_all(class_='cuadro_datos_grafico_viento_hora')]
direcciones = [elemento.text for elemento in soup.find_all(class_='cuadro_datos_grafico_viento_datos_direccion')]
alturas = [elemento.text for elemento in soup.find_all(class_='cuadro_datos_grafico_oleaje_datos_altura')]
periodos = [elemento.text for elemento in soup.find_all(class_='cuadro_datos_grafico_oleaje_datos_periodo')]

# Imprimir los valores obtenidos
print("Valores de las horas:", horas)
print("Valores de las direcciones:", direcciones)
print("Valores de las alturas:", alturas)
print("Valores de los periodos:", periodos)

# Encontrar los elementos deseados
elemento_salida_sol = soup.find(class_='l_grafico_mareas_responsive_ico_salida_sol')
elemento_puesta_sol = soup.find(class_='l_grafico_mareas_responsive_ico_puesta_sol')

# Extraer los valores de las horas de salida y puesta del sol
hora_salida_sol = elemento_salida_sol.find(class_='l_grafico_mareas_responsive_hora_salida_puesta_sol').text.strip()
hora_puesta_sol = elemento_puesta_sol.find(class_='l_grafico_mareas_responsive_hora_salida_puesta_sol').text.strip()

# Imprimir los valores obtenidos
print("Hora de salida del sol:", hora_salida_sol)
print("Hora de puesta del sol:", hora_puesta_sol)




Valores de las horas: ['0:00 h', '1:00 h', '2:00 h', '3:00 h', '4:00 h', '5:00 h', '6:00 h', '7:00 h', '8:00 h', '9:00 h', '10:00 h', '11:00 h', '12:00 h', '13:00 h', '14:00 h', '15:00 h', '16:00 h', '17:00 h', '18:00 h', '19:00 h', '20:00 h', '21:00 h', '22:00 h', '23:00 h']
Valores de las direcciones: ['NE', 'NE', 'NE', 'NE', 'NE', 'NE', 'NE', 'NE', 'NE', 'NE', 'NE', 'NE', 'NE', 'NE', 'NE', 'NE', 'NE', 'NE', 'NE', 'NE', 'NE', 'NE', 'NE', 'NE']
Valores de las alturas: ['0.1', '0.1', '0.1', '0.1', '0.1', '0.2', '0.2', '0.2', '0.2', '0.2', '0.2', '0.2', '0.2', '0.2', '0.3', '0.3', '0.4', '0.4', '0.5', '0.6', '0.6', '0.7', '0.7', '0.7']
Valores de los periodos: ['0 s', '0 s', '0 s', '0 s', '0 s', '0 s', '0 s', '0 s', '0 s', '0 s', '0 s', '0 s', '0 s', '0 s', '0 s', '0 s', '0 s', '0 s', '0 s', '0 s', '0 s', '0 s', '0 s', '0 s']
Hora de salida del sol: 8:08 h
Hora de puesta del sol: 17:30 h


## PLEAMARES Y BAJAMARES

In [24]:
# Buscar el div específico por el id y la clase
elemento = soup.find('div', {'id': 'brujula_mareas_texto_movil', 'class': 'grafico_estado_actual_texto2'})

# Verificar si se encontró el elemento
if elemento is not None:
    # Obtener el texto del elemento
    texto = elemento.text.strip()

    # Imprimir el resultado obtenido
    print(texto)
else:
    print("No se encontró el elemento especificado.")

El nivel del agua está bajando en estos momentos. La bajamar será dentro de 3 horas y 30 minutos.


In [25]:
# Buscar todos los div con la clase grafico_estado_actual_texto
divs_estado_actual = soup.find_all('div', class_='grafico_estado_actual_texto')

if divs_estado_actual:
    for div_estado_actual in divs_estado_actual:
        # Obtener el contenido de cada elemento div
        contenido = div_estado_actual.text.strip()
        
        # Filtrar el texto específico
        if 'Hasta la puesta de sol' in contenido:
            print(contenido)
else:
    print("No se encontraron elementos div con la clase grafico_estado_actual_texto.")


Hoy  jueves, 1 de junio de 2023, amaneció en Puerto Montt a las 8:08:32 h y la puesta de sol será a las 17:30:46 h. En el gráfico de pleamares y bajamares, podemos observar que la primera bajamar fue a las 5:31 h y la siguiente bajamar  será a las 18:04 h. La única pleamar de hoy  fue a las 11:44 h.  

 El nivel del agua está bajando en estos momentos. La bajamar será dentro de 3 horas y 30 minutos. 

 Hasta la puesta de sol de las 17:30:46 h, habremos tenido un total de 9 horas y 22 minutos de sol. El tránsito solar fue a las 12:49:39 h.


## COEFICIENTE DE MAREAS

In [26]:
valor_manana = soup.find('div', id='datos_grafico_triangulo_coeficiente_mañana').find('div', class_='datos_grafico_triangulo_coeficiente_numero').text.strip()
texto_manana = soup.find('div', id='datos_grafico_triangulo_coeficiente_mañana').find('div', class_='datos_grafico_triangulo_coeficiente_texto').text.strip()

# Obtener el valor de tarde
valor_tarde = soup.find('div', id='datos_grafico_triangulo_coeficiente_tarde').find('div', class_='datos_grafico_triangulo_coeficiente_numero').text.strip()
texto_tarde = soup.find('div', id='datos_grafico_triangulo_coeficiente_tarde').find('div', class_='datos_grafico_triangulo_coeficiente_texto').text.strip()

# Obtener el coeficiente de mareas
coeficiente_mareas = soup.find('div', class_='texto_grafico_estado_actual').find('strong').text.strip()

# Obtener el texto de fecha
texto_fecha = soup.find('div', class_='texto_grafico_estado_actual_fecha').text.strip()

print("Valor de mañana:", valor_manana)
print("Texto de mañana:", texto_manana)
print("Valor de tarde:", valor_tarde)
print("Texto de tarde:", texto_tarde)
print("Coeficiente de mareas:", coeficiente_mareas)
print("Texto de fecha:", texto_fecha)

Valor de mañana: 76
Texto de mañana: MAÑANA
Valor de tarde: 80
Texto de tarde: TARDE
Coeficiente de mareas: Temperatura actual   aire /  agua
Texto de fecha: 1 DE JUNIO DE 2023, 14:33 h


In [27]:
def filtrar_contenido(class_name, texto_buscar):
    divs_estado_actual = soup.find_all('div', class_=class_name)

    for div_estado_actual in divs_estado_actual:
        contenido = div_estado_actual.text.strip()
        if texto_buscar in contenido:
            return contenido

    return None

In [28]:
texto1 = filtrar_contenido('grafico_estado_actual_texto1', 'Podemos comparar estos niveles con la')
print(texto1)

Podemos comparar estos niveles con la pleamar máxima registrada en las tablas de mareas de Puerto Montt que es de 7,4 m y la altura mínima -0,1 m.


In [29]:
texto2 = filtrar_contenido('grafico_estado_actual_texto2', 'Tomando como referencia la')
print(texto2)

Tomando como referencia la bajamar media inferior (BMI), las alturas de las mareas de hoy son 1,5 m, 5,9 m y 1,4 m.


In [30]:
texto3 = filtrar_contenido('grafico_estado_actual_texto1', 'Hoy comenzamos el dia con un coeficiente de mareas')
print(texto3)

Hoy comenzamos el dia con un coeficiente de mareas de 76, un valor alto y por tanto la amplitud de la marea y las corrientes también serán altas. A mediodía el valor del coeficiente de mareas crece hasta 80 para acabar el día con un coeficiente de mareas de 83.


In [31]:
# Obtener los valores de coeficientes y horas
divs_coef = soup.find_all('div', class_='l_grafico_alturas_responsive_coef_datos_coef')
valores_coeficientes = [div.text.strip() for div in divs_coef]

divs_hora = soup.find_all('div', class_='l_grafico_alturas_responsive_coef_datos_hora')
valores_horas = [div.text.strip() for div in divs_hora]

# Obtener el valor 'coef.'
span_coef = soup.find_all('span', class_='coef_grafico_ocultar')
valor_coef_ocultar = [span.text.strip() for span in span_coef]



# Imprimir los valores obtenidos
print("Valores de coeficientes:", valores_coeficientes)
print("Valores de horas:", valores_horas)
print("Valor coef:", valor_coef_ocultar)


Valores de coeficientes: ['76', '80', '83']
Valores de horas: ['coef.  0:00 h', 'coef.  12:00 h', 'coef.  0:00 h']
Valor coef: ['coef.', 'coef.', 'coef.']


In [32]:
# Obtener el valor de l_grafico_alturas_responsive_altura_max
div_altura_max = soup.find('div', id='l_grafico_alturas_responsive_altura_max')
valor_altura_max = div_altura_max.text.strip()

# Obtener el valor de tabla_mareas_marea_altura_numero
div_altura_min = soup.find('div', id='l_grafico_alturas_responsive_altura_min')
valor_altura_min = div_altura_min.text.strip()

# Imprimir los valores obtenidos
print("Valor de l_grafico_alturas_responsive_altura_max:", valor_altura_max)
print("Valor de l_grafico_alturas_responsive_altura_min:", valor_altura_min)

Valor de l_grafico_alturas_responsive_altura_max: altura máx 7,4 m
Valor de l_grafico_alturas_responsive_altura_min: altura min -0,1 m


In [33]:
divs_punto_azul = soup.find_all('div', class_='l_grafico_mareas_responsive_punto_azul')
valores_punto_azul = []
for div_azul in divs_punto_azul:
    hora_azul_element = div_azul.find('div', class_='l_grafico_alturas_responsive_punto_azul_hora')
    globo_azul_element = div_azul.find('div', class_='l_grafico_alturas_responsive_punto_azul_hora_globo')
    if hora_azul_element and globo_azul_element:
        hora_azul = hora_azul_element.text.strip()
        globo_azul = globo_azul_element.text.strip()
        valores_punto_azul.extend([globo_azul, hora_azul])

# Obtener los valores de l_grafico_alturas_responsive_punto_rojo
divs_punto_rojo = soup.find_all('div', class_='l_grafico_mareas_responsive_punto_rojo')
valores_punto_rojo = []
for div_rojo in divs_punto_rojo:
    hora_rojo_element = div_rojo.find('div', class_='l_grafico_alturas_responsive_punto_rojo_hora')
    globo_rojo_element = div_rojo.find('div', class_='l_grafico_alturas_responsive_punto_rojo_hora_globo')
    if hora_rojo_element and globo_rojo_element:
        hora_rojo = hora_rojo_element.text.strip()
        globo_rojo = globo_rojo_element.text.strip()
        valores_punto_rojo.extend([globo_rojo, hora_rojo])

# Imprimir los valores obtenidos
print("Valores de l_grafico_alturas_responsive_punto_azul:", ', '.join(valores_punto_azul))
print("Valores de l_grafico_alturas_responsive_punto_rojo:", ', '.join(valores_punto_rojo))

Valores de l_grafico_alturas_responsive_punto_azul: 5,9, 11:44 h
Valores de l_grafico_alturas_responsive_punto_rojo: 1,5, 5:31 h, 1,4, 18:04 h


In [34]:
divs = soup.find_all('div', class_='l_grafico_coef_globo_punto')

valores = []
for div in divs:
    valor = div.find('span').text.strip()
    valores.append(valor)

print("Valores:", ', '.join(valores))

Valores: 76, 87, 60, 74, 46


## TABLA DE MAREAS

In [111]:
tabla_mareas_dia_values = []

# Encuentra todos los elementos con la clase "tabla_mareas_dia_numero"
elementos_dia = soup.find_all(class_="tabla_mareas_dia_numero")

# Extrae el texto de cada elemento y agrega los valores a la lista tabla_mareas_dia_values
for elemento in elementos_dia:
    valor = elemento.get_text(strip=True)
    tabla_mareas_dia_values.append(valor.strip())

max_dia = max(map(int, tabla_mareas_dia_values))
num_bucles = len(tabla_mareas_dia_values) // max_dia

tabla_mareas_marea_values = [[], [], [], []]
tabla_mareas_coeficiente_values = []
tabla_mareas_actividad_values = []

# Encuentra todos los elementos con la clase "tabla_mareas_marea"
elementos_marea = soup.find_all(class_="tabla_mareas_marea")

# Itera sobre los elementos de la lista
for i, elemento in enumerate(elementos_marea):
    # Extrae el texto del elemento
    valor = elemento.get_text(strip=True)
    # Determina a qué columna pertenece el elemento y agrega el valor a la lista correspondiente
    columna = i % 4
    tabla_mareas_marea_values[columna].append(valor)

# Encuentra todos los elementos con la clase "tabla_mareas_coeficiente_numero"
elementos_coeficiente = soup.find_all(class_="tabla_mareas_coeficiente_numero")

# Extrae el texto de cada elemento y agrega los valores a la lista tabla_mareas_coeficiente_values
for elemento in elementos_coeficiente:
    valor = elemento.get_text(strip=True)
    tabla_mareas_coeficiente_values.append(valor)

# Encuentra todos los elementos con la clase "tabla_mareas_actividad"
elementos_actividad = soup.find_all(class_="tabla_mareas_actividad")

# Itera sobre los elementos de la lista
for elemento in elementos_actividad:
    # Encuentra todos los elementos hijos con la clase "icon-ic_pez_leyenda2 noprint"
    elementos_icon = elemento.find_all(class_="icon-ic_pez_leyenda2 noprint")
    # Cuenta la cantidad de elementos encontrados y agrega el valor a la lista
    cantidad = len(elementos_icon)
    tabla_mareas_actividad_values.append(3 - cantidad)

# Obtén el máximo número de elementos en las columnas de marea
max_marea_length = max(len(tabla_mareas_marea_values[0]), len(tabla_mareas_marea_values[1]), len(tabla_mareas_marea_values[2]), len(tabla_mareas_marea_values[3]))


# Imprime los valores de las mareas de 4 en 4
for i in range(max_dia):
    print(f"DÍA: {tabla_mareas_dia_values[i]}")
    print(f"1ª MAREA: {tabla_mareas_marea_values[0][i]}")
    print(f"2ª MAREA: {tabla_mareas_marea_values[1][i]}")
    print(f"3ª MAREA: {tabla_mareas_marea_values[2][i]}")
    print(f"4ª MAREA: {tabla_mareas_marea_values[3][i]}")
    print(f"COEFICIENTE: {tabla_mareas_coeficiente_values[i]}")
    print(f"ACTIVIDAD MEDIA: {tabla_mareas_actividad_values[i]}")
    print("-----")


DÍA: 1
1ª MAREA: 5:31h1,5m
2ª MAREA: 11:44h5,9m
3ª MAREA: 18:04h1,4m
4ª MAREA: 
COEFICIENTE: 76alto
ACTIVIDAD MEDIA: 1
-----
DÍA: 2
1ª MAREA: 0:05h5,6m
2ª MAREA: 6:10h1,4m
3ª MAREA: 12:22h6,2m
4ª MAREA: 18:45h1,1m
COEFICIENTE: 83alto
ACTIVIDAD MEDIA: 2
-----
DÍA: 3
1ª MAREA: 0:47h5,7m
2ª MAREA: 6:49h1,3m
3ª MAREA: 13:01h6,5m
4ª MAREA: 19:26h0,9m
COEFICIENTE: 86alto
ACTIVIDAD MEDIA: 3
-----
DÍA: 4
1ª MAREA: 1:30h5,7m
2ª MAREA: 7:29h1,3m
3ª MAREA: 13:42h6,6m
4ª MAREA: 20:10h0,8m
COEFICIENTE: 87alto
ACTIVIDAD MEDIA: 3
-----
DÍA: 5
1ª MAREA: 2:13h5,7m
2ª MAREA: 8:11h1,3m
3ª MAREA: 14:25h6,6m
4ª MAREA: 20:55h0,9m
COEFICIENTE: 85alto
ACTIVIDAD MEDIA: 2
-----
DÍA: 6
1ª MAREA: 3:00h5,6m
2ª MAREA: 8:56h1,5m
3ª MAREA: 15:11h6,5m
4ª MAREA: 21:44h1,0m
COEFICIENTE: 80alto
ACTIVIDAD MEDIA: 1
-----
DÍA: 7
1ª MAREA: 3:50h5,4m
2ª MAREA: 9:45h1,6m
3ª MAREA: 16:01h6,3m
4ª MAREA: 22:38h1,2m
COEFICIENTE: 74alto
ACTIVIDAD MEDIA: 0
-----
DÍA: 8
1ª MAREA: 4:46h5,3m
2ª MAREA: 10:41h1,9m
3ª MAREA: 16:58h6,0m
4ª

## CABALLO DE BATALLA

In [41]:
# Buscar todos los div con la clase grafico_estado_actual_texto
divs_estado_actual = soup.find_all('div', class_='grafico_estado_actual_texto')

if divs_estado_actual:
    for div_estado_actual in divs_estado_actual:
        # Obtener el contenido de cada elemento div
        contenido = div_estado_actual.text.strip()
        
        # Imprimir el resultado obtenido
        print(contenido)
else:
    print("No se encontraron elementos div con la clase grafico_estado_actual_texto.")

En este momento la temperatura actual del agua en Puerto Montt es de 11°C. La temperatura media del agua en Puerto Montt en el dia de hoy es de 10°C.                    

  La temperatura del agua juega un papel determinante en la conducta de los peces. Cuando el agua está fría los peces se muestran aletargados e inactivos, y lo mismo ocurre si el agua está demasiado caliente.
Hoy  jueves, 1 de junio de 2023, amaneció en Puerto Montt a las 8:08:32 h y la puesta de sol será a las 17:30:46 h. En el gráfico de pleamares y bajamares, podemos observar que la primera bajamar fue a las 5:31 h y la siguiente bajamar  será a las 18:04 h. La única pleamar de hoy  fue a las 11:44 h.  

 El nivel del agua está bajando en estos momentos. La bajamar será dentro de 3 horas y 30 minutos. 

 Hasta la puesta de sol de las 17:30:46 h, habremos tenido un total de 9 horas y 22 minutos de sol. El tránsito solar fue a las 12:49:39 h.
Hoy comenzamos el dia con un coeficiente de mareas de 76, un valor alto y p